# Awesome Github Actions
> Try Github Actions with Node.js and Python projects.

- toc: false 
- badges: true
- comments: true
- categories: [github, node.js, python]

## Deploy Github Pages with Github Actions

Just weeks ago, I set up this blog with Jekyll on Github Pages. However, I switched to [fastpages](https://github.com/fastai/fastpages) because of Jupyter Notebook support. The swithcing isn't that difficult, but something caught my attention during the process, and that is [Github Actions](https://github.com/features/actions). We have talked about the idea that only pushing source files to the github repo and let Github Pages generate static files by its own. But Github Pages only supports Jekyll and limited number of plugins, and that is where Github Actions can help us.

Basically, Github Actions is like a remote server that can execute some predifined commands. To use Github Actions to deploy our Github Pages, we still need to generate static files in our repo, but the generation can be done automatically by Github Actions, and generated files can be hosted in another branch other than master. Therefore, we can host all source files in the master branch, and every time we push commits to the master branch (e.g. new posts or modification of layouts), the predifined Github Actions workflow is triggered and it will automatically generate all necessary static files in another branch. We just need to change the repo's setting that decides which branch Github Pages site is built from.

Take [my homepage repo](https://github.com/graysonliu/graysonliu.github.io) as an example. I build it using React and Webpack from scratch. Webpack will generate static files in a folder called `dist`. This dist folder does not have to be in the master branch, but all files inside it should be the content of another branch called `gh-pages`, and the Github Pages is built from this branch.

There are existing Github Actions tools to help us achieve that for Github Pages. I chose the one that fastpages uses, which is [peaceiris/actions-gh-pages](https://github.com/peaceiris/actions-gh-pages). The usage is actually pretty straightforward, the entire workflow is as follows:
```yaml
name: build

on:
  push:
    branches:
      - master

jobs:
  deploy:
    runs-on: ubuntu-18.04
    steps:
      - uses: actions/checkout@v2

      - name: Setup Node
        uses: actions/setup-node@v2.1.0
        with:
          node-version: '12.x'

      - name: Cache dependencies
        uses: actions/cache@v2
        with:
          path: ~/.npm
          key: ${{ runner.os }}-node-${{ hashFiles('**/package-lock.json') }}
          restore-keys: |
            ${{ runner.os }}-node-
      - run: npm ci
      - run: npm run build

      - name: Deploy
        uses: peaceiris/actions-gh-pages@v3
        with:
          github_token: ${{ secrets.GITHUB_TOKEN }}
          publish_dir: ./dist
```

At first, we give a name to the workflow, which is `build`. Then, we specify what will trigger this workflow, which is pushing commits to the master branch.